# 類似画像検索のサンプルノートブック
このノートブックは、機械学習と Amazon Elasticsearch Service (Amazon ES) を使って類似画像検索を行うサンプルノートブックです。このノートブックは、conda_mxnet_p36 カーネルで実行してください。<br>
[こちら](https://aws.amazon.com/jp/builders-flash/202102/elasticsearch-your-cat/) の記事の手順で Amazon Elasticsearch Service と Kibana の設定が完了してからこのノートブックを実行してください。

## パスの設定
このノートブックを実行する前に、ご自身の環境に合わせて以下を設定してください。

* `es_host` に、Amazon Elasticsearch Service のドメインのエンドポイント名を記載します。<br>
* `region` に、このノートブックインスタンスのリージョンを記載します。

In [ ]:
es_host = 'esdomainname.ap-northeast-1.es.amazonaws.com'
region = 'ap-northeast-1' # e.g. us-west-1

## 実行環境の設定
このサンプルでは、画像を特徴ベクトルに変換するために MXNet Gluon の学習済みの機械学習モデルを使用するため、gluoncv をインストールします。

In [ ]:
!pip install gluoncv

In [ ]:
import mxnet as mx
from mxnet import gluon, nd
from mxnet.gluon.model_zoo import vision
import multiprocessing
from mxnet.gluon.data.vision.datasets import ImageFolderDataset
from mxnet.gluon.data import DataLoader
import numpy as np
# import wget
import imghdr
import json
import pickle
import numpy as np
import glob, os, time
import matplotlib.pyplot as plt 
import matplotlib.gridspec as gridspec
import urllib.parse
import urllib
import gzip
import os
import tempfile
import glob
from os.path import join
%matplotlib inline

## 機械学習モデルの設定
このサンプルでは、画像から特徴ベクトルに変換するために学習済みの機械学習モデルを使用します。
ここでは、MXNet の model-zoo のモデルを使用します。model-zoo のネットワークは、特徴量が .features プロパティにあり、出力が .output プロパティにあります。この仕組みを利用して、事前にトレーニングされたネットワークを使って featurizer を非常に簡単に作成できます。

In [ ]:
BATCH_SIZE = 256
EMBEDDING_SIZE = 512
SIZE = (224, 224)
MEAN_IMAGE= mx.nd.array([0.485, 0.456, 0.406])
STD_IMAGE = mx.nd.array([0.229, 0.224, 0.225])

In [ ]:
ctx = mx.gpu() if len(mx.test_utils.list_gpus()) else mx.cpu()
net = vision.resnet18_v2(pretrained=True, ctx=ctx).features
net.hybridize()

## データセットの準備
### データの変換
モデルの入力サイズに合わせるため、元画像をリサイズとクロップしてサイズを 224 x 224 にして画素値を 0 から 1 に正規化します。

In [ ]:
def transform(image, label):
    resized = mx.image.resize_short(image, SIZE[0]).astype('float32')
    cropped, crop_info = mx.image.center_crop(resized, SIZE)
    cropped /= 255.
    normalized = mx.image.color_normalize(cropped,
                                      mean=MEAN_IMAGE,
                                      std=STD_IMAGE) 
    transposed = nd.transpose(normalized, (2,0,1))
    return transposed, label

検索対象となる画像を展開します。[こちら](https://d1.awsstatic.com/Developer%20Marketing/jp/magazine/sample/data_elasticsearch-cat.2a73bad33290a2d7d909235c6a963f8fd3da691a.zip) の zip ファイルをダウンロード、解凍すると images.zip を取得できます。取得した images.zip をこのノートブックと同じ場所にアップロードしてから以降のセルを実行してください。

In [ ]:
image_path = './images'

In [ ]:
!unzip images.zip

ここで、猫の写真が入った images フォルダが作成されました。Jupyter のファイルブラウザをご確認ください。みなさまの猫の写真を追加する場合は、こちらの images フォルダに JPEG 画像を追加していただくと、みなさまの猫の写真が検索対象になります。写真を追加せずこのまま進めても構いません。

In [ ]:
empty_folder = tempfile.mkdtemp()
# Create an empty image Folder Data Set
dataset = ImageFolderDataset(root=empty_folder, transform=transform)

In [ ]:
list_files = glob.glob(os.path.join(image_path, '**.jpg') , recursive=True)
print("[{}] images".format(len(list_files)))

In [ ]:
dataset.items = list(zip(list_files, [0]*len(list_files)))
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, last_batch='keep', shuffle=False, num_workers=multiprocessing.cpu_count())

## 画像から特徴ベクトルに変換
機械学習モデルを使って画像を特徴ベクトルに変換します。

In [ ]:
features = np.zeros((len(dataset), EMBEDDING_SIZE), dtype=np.float32)

次のセルの実行が完了するまでは t2.medium のインスタンスタイプで 20〜30秒ほどかかります。

In [ ]:
%%time
tick = time.time()
n_print = 100
j = 0
for i, (data, label) in enumerate(dataloader):
    data = data.as_in_context(ctx)
    if i%n_print == 0 and i > 0:
        print("{0} batches, {1} images, {2:.3f} img/sec".format(i, i*BATCH_SIZE, BATCH_SIZE*n_print/(time.time()-tick)))
        tick = time.time()
    output = net(data)
    features[(i)*BATCH_SIZE:(i+1)*max(BATCH_SIZE, len(output)), :] = output.asnumpy().squeeze()

## Amazon Elasticsearch Service のセットアップ
このサンプルでは、Amazon Elasticsearch Service (Amazon ES) を使って類似ベクトルを検索します。
ここでは、Amazon ES を使うための設定をします。

In [ ]:
!pip install elasticsearch requests_aws4auth

### Amazon ES インデックスの作成と特徴ベクトルの登録

In [ ]:
import time
import math
import numpy as np
import json
import certifi
from elasticsearch import Elasticsearch, helpers, RequestsHttpConnection
from sklearn.preprocessing import normalize
from requests_aws4auth import AWS4Auth
import elasticsearch
import boto3

dim = EMBEDDING_SIZE
fvecs = features

np.shape(fvecs)

service = 'es'
credentials = boto3.Session().get_credentials()
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, region, service, session_token=credentials.token)

idx_name = 'vsearch'

es = Elasticsearch(
    hosts = [{'host': es_host, 'port': 443}],
    http_auth = awsauth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection
)

res = es.cluster.put_settings({'persistent': {'knn.algo_param.index_thread_qty': 2}})
print(res)

mapping = {
    "settings" : {
        "index" : {
            "knn": True,
            "knn.algo_param" : {
                "ef_search" : "256",
                "ef_construction" : "128",
                "m" : "48"
            },
            'refresh_interval': -1,
            'translog.flush_threshold_size': '10gb',
            'number_of_replicas': 0
        }
    },
    'mappings': {
        'properties': {
            'fvec': {
                'type': 'knn_vector',
                'dimension': dim
            }
        }
    }
}

res = es.indices.create(index=idx_name, body=mapping, ignore=400)
print(res)

bs = 100
nloop = math.ceil(fvecs.shape[0] / bs)

for k in range(nloop):
    rows = [{'_index': idx_name, '_id': f'{i}',
             '_source': {'fvec': normalize(fvecs[i:i+1])[0].tolist()}}
             for i in range(k * bs, min((k + 1) * bs, fvecs.shape[0]))]
    s = time.time()
    helpers.bulk(es, rows, request_timeout=30)
#     print(k, time.time() - s)
    
res = es.indices.refresh(index=idx_name)

インデックス一覧を表示して、インデックスが作成されたか確認します。

In [ ]:
res = es.cat.indices(v=True)
item_num=es.count(index=idx_name)['count']
print(res)
print('number of images:', item_num)

[ご参考] インデックスに新しい特徴ベクトルを追加する場合はコメントアウトを外して以下のコードを実行します。

In [ ]:
# image_path = 'xxx.jpg'
# image = plt.imread(image_path)[:,:,:3]
# image_t, _ = transform(nd.array(image), 1) # モデルに合わせて画像サイズを変換
# output = net(image_t.expand_dims(axis=0).as_in_context(ctx)) # 画像の特徴ベクトルを取得
# item_num=es.count(index=idx_name)['count']
# body={'fvec': normalize(output.asnumpy())[0].tolist()}
# es.index(idx_name, body=body, id=item_num)
# res = es.indices.refresh(index=idx_name)
# item_num=es.count(index=idx_name)['count']

# # dataset 更新
# dataset.items.append((image_path, 0))

[ご参考] ID を指定してインデックスからアイテム（ドキュメント）を削除する場合はコメントアウトを外して以下のコードを実行します。

In [ ]:
#  任意の ID を持つドキュメントがあるか確認（以下の例では id=4）
# del_id = 4
# es.get(index=idx_name,id=del_id)
# es.delete(index=idx_name, id=del_id)
# res = es.indices.refresh(index=idx_name)

## 画像の検索
ここでは、作成した Amazon ES を使った類似画像検索を行います。まずは検索および検索結果表示のための関数を定義します。

In [ ]:
def search_image(feature, num):
    start = time.time()
    res = es.search(request_timeout=300, index=idx_name,
                    body={'size': num, '_source': False,
                          'query': {'knn': {'fvec': {'vector': normalize(feature)[0].tolist(), 'k': num}}}})
    print('time for query: ', str((time.time()-start)*1000), 'msec')
#     print(json.dumps(res, indent=2))
    return res

In [ ]:
def plot_predictions(images, scores):
    rows = len(images)//3+2
    gs = gridspec.GridSpec(rows, 3)
    fig = plt.figure(figsize=(15, 6*rows))
    gs.update(hspace=0.1, wspace=0.1)
    for i, (gg, image) in enumerate(zip(gs, images)):
        gg2 = gridspec.GridSpecFromSubplotSpec(10, 10, subplot_spec=gg)
        ax = fig.add_subplot(gg2[:,:])
        ax.imshow(image, cmap='Greys_r')
        ax.tick_params(axis='both',       
                       which='both',      
                       bottom='off',      
                       top='off',         
                       left='off',
                       right='off',
                       labelleft='off',
                       labelbottom='off') 
        ax.axes.set_title("result [{}] score:{}".format(i, scores[i-1]))
        if i == 0:
            plt.setp(ax.spines.values(), color='red')
            ax.axes.set_title("SEARCH".format(i))

### ランダムに画像を選んで画像検索
サンプルデータセットの中からランダムに一枚ピックアップして、その画像と似ている画像を表示します。

In [ ]:
index = np.random.randint(0,item_num)
print('image index:', index)
res = search_image(fvecs[index:index+1], 10)

idx = []
scores = []
for i in res['hits']['hits']:
    idx.append(int(i['_id']))
    scores.append(i['_score'])
    
    
# images = [plt.imread(dataset.items[index][0])]
images=[]
images += [plt.imread(dataset.items[label][0]) for label in idx[:]]
plot_predictions(images, scores)

### 任意の画像と似ている画像を検索
ノートブックインスタンスに画像をアップロードし、その画像に似た画像を検索してみましょう。`testimg`に検索のキーとなる画像のパスを記載してから以下のセルを実行します。

In [ ]:
testimg = '<file_path>' #  ./images/input.jpg のように画像を指定
image = plt.imread(testimg)[:,:,:3]
image_t, _ = transform(nd.array(image), 1) # モデルに合わせて画像サイズを変換
output = net(image_t.expand_dims(axis=0).as_in_context(ctx)) # 画像の特徴ベクトルを取得
res = search_image(output.asnumpy(), 10)

idx = []
scores = []
for i in res['hits']['hits']:
    idx.append(int(i['_id']))
    scores.append(i['_score'])
    
images = [image]
images += [plt.imread(dataset.items[label][0]) for label in idx[:]]
plot_predictions(images, scores)

## リソースの削除
[こちらの記事](https://aws.amazon.com/jp/builders-flash/202102/elasticsearch-your-cat/) の「リソースの削除」の部分を参考に、今回のハンズオンで作成したリソースを削除して課金を停止しましょう。この手順を忘れると課金が発生し続けますのでご注意ください。

## まとめ
このノートブックでは、機械学習と Amazon ES を使った類似画像検索の方法をご紹介しました。今回は MXNet Gluon の学習済みモデルの中間出力を特徴ベクトルとして Amazon ES に登録し、K-NN を使って類似する特徴ベクトルを検索するアプローチを採りました。検索精度をあげる場合は、写真の撮影条件（明るさ、被写体の大きさ、ホワイトバランスなど）を揃える、特徴ベクトルを作成する部分のアルゴリズムを変更する、特徴ベクトルの次元を増やすなどの方法が考えられます。